## 다중 선형 회귀

In [20]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader #데이터세트와 데이터로더 사용위해 임포트


train_x = torch.FloatTensor([
    [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
    [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

In [21]:
train_dataset = TensorDataset(train_x, train_y)  #다변량 다중 선형 회귀를 계산하기 위해 해당 형태로 독립변수, 종속변수 입력
#배치 크기를 2로 선언, 데이터 순서 변경을 참값으로 적용해 순서를 무작위로 변경, 배치 크기에 맞지 않는 배치 제거
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

In [22]:
model = nn.Linear(2, 2, bias=True) #입출력 데이터 차원의 크기는 2
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [23]:
for epoch in range(20000):
    cost = 0.0 # 에폭마다 오차를 다시 계산하기 위해 초기화

    for batch in train_dataloader: #오차가 아닌 손실을 계산함
        x, y = batch
        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[0.3287, 0.4243],
        [0.2066, 1.0288]], requires_grad=True), Parameter containing:
tensor([-0.6050, -0.5565], requires_grad=True)], Cost : 0.044
Epoch : 2000, Model : [Parameter containing:
tensor([[0.4780, 0.3471],
        [0.2722, 0.9949]], requires_grad=True), Parameter containing:
tensor([-0.8315, -0.6561], requires_grad=True)], Cost : 0.012
Epoch : 3000, Model : [Parameter containing:
tensor([[0.5541, 0.3077],
        [0.3057, 0.9775]], requires_grad=True), Parameter containing:
tensor([-0.9470, -0.7069], requires_grad=True)], Cost : 0.003
Epoch : 4000, Model : [Parameter containing:
tensor([[0.5929, 0.2877],
        [0.3227, 0.9688]], requires_grad=True), Parameter containing:
tensor([-1.0058, -0.7327], requires_grad=True)], Cost : 0.001
Epoch : 5000, Model : [Parameter containing:
tensor([[0.6126, 0.2775],
        [0.3314, 0.9642]], requires_grad=True), Parameter containing:
tensor([-1.0357, -0.7459], requires_grad=True)]

## 편향 제거

In [24]:
train_x = torch.FloatTensor([
    [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
    [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

In [25]:
train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

In [26]:
model = nn.Linear(2, 2, bias=False)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [27]:
for epoch in range(20000):
    cost = 0.0

    for batch in train_dataloader: #오차가 아닌 손실을 계산
        x, y = batch
        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.8144, -0.0836],
        [ 0.9769,  0.2996]], requires_grad=True)], Cost : 0.051
Epoch : 2000, Model : [Parameter containing:
tensor([[ 0.9961, -0.2307],
        [ 1.0021,  0.2792]], requires_grad=True)], Cost : 0.032
Epoch : 3000, Model : [Parameter containing:
tensor([[ 1.1405, -0.3477],
        [ 1.0222,  0.2629]], requires_grad=True)], Cost : 0.020
Epoch : 4000, Model : [Parameter containing:
tensor([[ 1.2554, -0.4405],
        [ 1.0382,  0.2500]], requires_grad=True)], Cost : 0.013
Epoch : 5000, Model : [Parameter containing:
tensor([[ 1.3466, -0.5143],
        [ 1.0509,  0.2397]], requires_grad=True)], Cost : 0.008
Epoch : 6000, Model : [Parameter containing:
tensor([[ 1.4192, -0.5729],
        [ 1.0610,  0.2316]], requires_grad=True)], Cost : 0.005
Epoch : 7000, Model : [Parameter containing:
tensor([[ 1.4768, -0.6195],
        [ 1.0690,  0.2251]], requires_grad=True)], Cost : 0.003
Epoch : 8000, Model : [Parameter containi

## 비선형 회귀

In [28]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]]) #결괏값이 이차 방정식형태라서
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [29]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1) #선형변환함수

    def forward(self, x):
        x = self.layer(x)
        return x

In [30]:
train_dataset = CustomDataset("non_linear.csv") #csv파일입력
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device) #평균 제곱 오차를 할당
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [32]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1060, -1.6996]], requires_grad=True), Parameter containing:
tensor([0.1573], requires_grad=True)], Cost : 0.129
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1049, -1.7029]], requires_grad=True), Parameter containing:
tensor([0.1859], requires_grad=True)], Cost : 0.117
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1049, -1.7030]], requires_grad=True), Parameter containing:
tensor([0.2120], requires_grad=True)], Cost : 0.102
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1044, -1.7028]], requires_grad=True), Parameter containing:
tensor([0.2361], requires_grad=True)], Cost : 0.100
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1040, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.2580], requires_grad=True)], Cost : 0.110
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.1036, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.2780], requires_grad=True)],

In [33]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.7426],
        [ 69.3994],
        [357.0559]])


In [34]:
torch.save(
    model,
    "model.pt"
)

In [35]:
torch.save(
    model.state_dict(),
    "model_state_dict.pt"
)

## 검증용 데이터세트를 통한 평가

In [36]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [37]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [38]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [39]:
dataset = CustomDataset("non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20


In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [41]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0977, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.5894], requires_grad=True)], Cost : 0.086
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1007, -1.7036]], requires_grad=True), Parameter containing:
tensor([0.5305], requires_grad=True)], Cost : 0.082
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.0997, -1.7035]], requires_grad=True), Parameter containing:
tensor([0.5061], requires_grad=True)], Cost : 0.081
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.0987, -1.7034]], requires_grad=True), Parameter containing:
tensor([0.4962], requires_grad=True)], Cost : 0.079
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0997, -1.7035]], requires_grad=True), Parameter containing:
tensor([0.4920], requires_grad=True)], Cost : 0.081
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.1002, -1.7036]], requires_grad=True), Parameter containing:
tensor([0.4902], requires_grad=True)],

In [42]:
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

X : tensor([[ 1.4400,  1.2000],
        [31.3600,  5.6000],
        [72.2500, -8.5000],
        [ 7.2900,  2.7000]])
Y : tensor([[  2.8300],
        [ 88.2700],
        [238.8400],
        [ 18.2600]])
Outputs : tensor([[  2.9088],
        [ 88.1615],
        [238.9327],
        [ 18.4878]], grad_fn=<AddmmBackward0>)
--------------------
X : tensor([[ 7.5690e+01, -8.7000e+00],
        [ 1.0000e-02, -1.0000e-01],
        [ 1.4400e+00, -1.2000e+00],
        [ 2.9160e+01,  5.4000e+00]])
Y : tensor([[250.0600],
        [  0.6800],
        [  6.6500],
        [ 81.9000]])
Outputs : tensor([[249.9369],
        [  0.6904],
        [  6.9970],
        [ 81.6825]], grad_fn=<AddmmBackward0>)
--------------------
X : tensor([[56.2500, -7.5000],
        [ 0.3600,  0.6000],
        [47.6100, -6.9000],
        [17.6400, -4.2000]])
Y : tensor([[187.6300],
        [  0.2600],
        [160.0900],
        [ 62.6700]])
Outputs : tensor([[187.6316],
        [  0.5830],
        [159.8268],
        [ 62.324

## 모델 불러오기

In [43]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("model.pt", map_location=device) #모델을 불러올 때 적용하려는 장치 상태의미
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


<ipython-input-44-f2f2ba7b7f4f>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pt", map_location=device)


In [45]:
with torch.no_grad(): #테스트 데이터세트나 임의의 값으로 모델을 확인하거나 평가할 때 사용함
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.7426],
        [ 69.3994],
        [357.0559]])


## 모델 구조 확인

In [46]:
class CustomModel(nn.Module):
    pass

In [47]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


<ipython-input-47-f2f2ba7b7f4f>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model.pt", map_location=device)


## 모델 상태 저장

In [48]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [49]:
train_dataset = CustomDataset("non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [50]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1011, -1.7003]], requires_grad=True), Parameter containing:
tensor([0.4172], requires_grad=True)], Cost : 0.081
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7033]], requires_grad=True), Parameter containing:
tensor([0.4238], requires_grad=True)], Cost : 0.075
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7033]], requires_grad=True), Parameter containing:
tensor([0.4298], requires_grad=True)], Cost : 0.078
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.4351], requires_grad=True)], Cost : 0.077
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1010, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.4401], requires_grad=True)], Cost : 0.072
Epoch : 6000, Model : [Parameter containing:
tensor([[ 3.1010, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.4446], requires_grad=True)],

In [51]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.8568],
        [ 69.4565],
        [356.8883]])


In [52]:
torch.save(
    model.state_dict(),
    "model_state_dict.pt"
)

## 모델 상태 불러오기

In [53]:
class CustomModel(nn.Module): #이 클러스를 선언하지 않으면, AtrributeError 오류 발생함
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

model_state_dict = torch.load("model_state_dict.pt", map_location=device)
model.load_state_dict(model_state_dict)

<ipython-input-54-589c87b2dd3a>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load("model_state_dict.pt", map_location=device)


<All keys matched successfully>

In [55]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)

## 체크 포인트 저장

In [56]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [57]:
train_dataset = CustomDataset("non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [58]:
checkpoint = 1
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
      #모델 저장 함수
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1

In [59]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.5670],
        [ 69.2779],
        [357.1521]])


## 체크포인트 불러오기

In [60]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [61]:
train_dataset = CustomDataset("non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [63]:
checkpoint = torch.load("checkpoint-6.pt")
model.load_state_dict(checkpoint["model_state_dict"]) #모델 불러올 때 사용
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

CustomModel 체크포인트-6


<ipython-input-63-715168366109>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("checkpoint-6.pt")


In [64]:
for epoch in range(checkpoint_epoch + 1, 10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 7000, Model : [Parameter containing:
tensor([[ 3.1071, -1.7026]], requires_grad=True), Parameter containing:
tensor([0.0643], requires_grad=True)], Cost : 0.163
Epoch : 8000, Model : [Parameter containing:
tensor([[ 3.1062, -1.7029]], requires_grad=True), Parameter containing:
tensor([0.1008], requires_grad=True)], Cost : 0.152
Epoch : 9000, Model : [Parameter containing:
tensor([[ 3.1056, -1.7027]], requires_grad=True), Parameter containing:
tensor([0.1343], requires_grad=True)], Cost : 0.127
Epoch : 10000, Model : [Parameter containing:
tensor([[ 3.1053, -1.7026]], requires_grad=True), Parameter containing:
tensor([0.1649], requires_grad=True)], Cost : 0.138


## 이진 분류

In [65]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length

In [66]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential( # 시퀀셜로 여러계층을 하나로 묶음
          nn.Linear(3, 1),
          nn.Sigmoid() #활성화함수로 시그모이드사용
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [67]:
dataset = CustomDataset("binary.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [68]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [69]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], requires_grad=True), Parameter containing:
tensor([0.0949], requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], requires_grad=True), Parameter containing:
tensor([-0.0346], requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], requires_grad=True), Parameter containing:
tensor([-0.1609], requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], requires_grad=True), Parameter containing:
tensor([-0.2843], requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], requires_grad=True), Parameter containing:
tensor([-0.4046], requires_grad=True)], Cost : 0.627
Epoch : 6000, Model : [Parameter containing:
tensor([[0.0065, 0.0034, 0.0068]], requires_grad=True), Parameter contain

In [70]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)

        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------")

tensor([[0.6707],
        [0.7298],
        [0.7368],
        [0.7800]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
tensor([[0.5723],
        [0.6746],
        [0.4676],
        [0.3640]])
tensor([[ True],
        [ True],
        [False],
        [False]])
--------------------
tensor([[0.5711],
        [0.6363],
        [0.3985],
        [0.6238]])
tensor([[ True],
        [ True],
        [False],
        [ True]])
--------------------
tensor([[0.6304],
        [0.7174],
        [0.5751],
        [0.6137]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
tensor([[0.7460],
        [0.6951],
        [0.7555],
        [0.6498]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
tensor([[0.8068],
        [0.7557],
        [0.6456],
        [0.7380]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
tensor([[0.4245],
        [0.7069],
  

## 모델 구조와 초깃값

In [71]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

        self.layer1[0].weight.data = torch.nn.Parameter(
            torch.Tensor([[0.4352, 0.3545],
                         [0.1951, 0.4835]])
        )

        self.layer1[0].bias.data = torch.nn.Parameter(
            torch.Tensor([-0.1419,  0.0439])
        )

        self.layer2[0].weight.data = torch.nn.Parameter(
            torch.Tensor([[-0.1725,  0.1129]])
        )

        self.layer2[0].bias.data = torch.nn.Parameter(
            torch.Tensor([-0.3043])
        )

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1)

## 단층 퍼셉트론 구조

In [72]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [73]:
train_dataset = CustomDataset("perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [74]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [75]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

<ipython-input-72-b746038c3bc6>:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-72-b746038c3bc6>:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.692
Epoch : 4000, Cost : 0.692
Epoch : 5000, Cost : 0.692
Epoch : 6000, Cost : 0.692
Epoch : 7000, Cost : 0.692
Epoch : 8000, Cost : 0.693
Epoch : 9000, Cost : 0.692
Epoch : 10000, Cost : 0.692


In [76]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.4667],
        [0.4980],
        [0.5032],
        [0.5344]])
tensor([[ True],
        [ True],
        [False],
        [False]])


# 다층 퍼셉트론 구조

In [77]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x


In [78]:
train_dataset = CustomDataset("perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [79]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [80]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

<ipython-input-77-44d2a935d29f>:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-77-44d2a935d29f>:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.681
Epoch : 2000, Cost : 0.533
Epoch : 3000, Cost : 0.403
Epoch : 4000, Cost : 0.109
Epoch : 5000, Cost : 0.046
Epoch : 6000, Cost : 0.028
Epoch : 7000, Cost : 0.020
Epoch : 8000, Cost : 0.015
Epoch : 9000, Cost : 0.012
Epoch : 10000, Cost : 0.010


In [81]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.0077],
        [0.9907],
        [0.9907],
        [0.0156]])
tensor([[ True],
        [False],
        [False],
        [ True]])
